In [10]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers, CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
import pandas as pd
import numpy as np
import imp, sys
sys.path.append('./src/')
import feature_utils
import seaborn as sns
import doc_utils_clean as doc_utils

In [22]:
take_columns = ['sent_len','first_sent_in_nar','last_sent_in_nar','is_client','is_nar']

In [23]:
imp.reload(doc_utils)
!pwd
sent_db = pd.read_csv("./dataframes/sent_db.csv",usecols=take_columns)

/path

In [24]:
sent_db.head()

,is_nar,sent_len,first_sent_in_nar,last_sent_in_nar,is_client
0,0.000000,10.000000,False,False,0
1,0.000000,15.000000,False,False,1
2,0.000000,4.000000,False,False,0
3,0.000000,8.000000,False,False,0
4,0.000000,73.000000,False,False,1


In [26]:
sent_tokens_db = pd.read_csv("./dataframes/sent_tokens_db_1.0.csv")

In [27]:
sent_tokens_db.head()

,FROM,sent_idx,TO,FORM,LEMMA,CPOSTAG,POSTAG,f_gen,f_num,FEATS,TOKEN,f_suf_gen,f_suf_num,f_suf_per,f_per,f_tense
0,0,0.000000,1,טוב,טוב,NNT,NNT,M,S,gen=M|num=S,1,NaN,NaN,NaN,NaN,NaN
1,1,0.000000,2,אהלן,אהלן,INTJ,INTJ,NaN,NaN,_,2,NaN,NaN,NaN,NaN,NaN
2,2,0.000000,3,נ,נ,NNP,NNP,M,S,gen=F|gen=M|num=S,3,NaN,NaN,NaN,NaN,NaN
3,0,1.000000,1,היי,הי,INTJ,INTJ,NaN,NaN,_,1,NaN,NaN,NaN,NaN,NaN
4,1,1.000000,2,מה,מה,QW,QW,NaN,NaN,_,2,NaN,NaN,NaN,NaN,NaN


In [86]:
# Feature set
def word2features(sent_idx,word_idx):
    sent_len = sent_tokens_db[sent_tokens_db['sent_idx']==sent_idx]['TOKEN'].max()
    word_row = sent_tokens_db.query("sent_idx==  @sent_idx and TOKEN == @word_idx")

    features = {
#         'bias': 1.0,
        'word':word_row['LEMMA'].item(),
        'postag': word_row['POSTAG'].item(),
        'gen': word_row['f_suf_gen'].item(),
        'num': word_row['f_suf_num'].item(),
        'suf_per': word_row['f_suf_per'].item(),
        'per': word_row['f_per'].item(),
        'tense': word_row['f_tense'].item(),
    }
    if word_idx > 1:
        one_before_idx = word_idx-1
        word_row = sent_tokens_db.query("sent_idx==  @sent_idx and TOKEN == @one_before_idx")
        features.update({
            '-1:word':word_row['LEMMA'].item(),
        '-1:postag': word_row['POSTAG'].item(),
        '-1:gen': word_row['f_suf_gen'].item(),
        '-1:num':word_row['f_suf_num'].item(),
        '-1:suf_per': word_row['f_suf_per'].item(),
        '-1:per': word_row['f_per'].item(),
        '-1:tense': word_row['f_tense'].item(),
        })
    else:
        features['BOS'] = True
    
    if word_idx > 2:
        two_before_idx = word_idx-2
        word_row = sent_tokens_db.query("sent_idx==  @sent_idx and TOKEN == @two_before_idx")
        features.update({
            '-2:word':word_row['LEMMA'].item(),
        '-2:postag': word_row['POSTAG'].item(),
        '-2:gen':word_row['f_suf_gen'].item(),
        '-2:num':word_row['f_suf_num'].item(),
        '-2:suf_per': word_row['f_suf_per'].item(),
        '-2:per': word_row['f_per'].item(),
        '-2:tense': word_row['f_tense'].item(),
        })

    if word_idx < sent_len:
        one_after_idx = word_idx+1
        word_row =  sent_tokens_db.query("sent_idx==  @sent_idx and TOKEN == @one_after_idx")
        features.update({
            '+1:word':word_row['LEMMA'].item(),
        '+1:postag': word_row['POSTAG'].item(),
        '+1:gen': word_row['f_suf_gen'].item(),
        '+1:num': word_row['f_suf_num'].item(),
        '+1:suf_per': word_row['f_suf_per'].item(),
        '+1:per': word_row['f_per'].item(),
        '+1:tense': word_row['f_tense'].item(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent_idx):
    return [word2features(sent_idx,word_idx) for word_idx in sent_tokens_db[sent_tokens_db['sent_idx']==sent_idx]['TOKEN']]


# def sent2labels(sent):
#     return [sent_tokens_db.iloc[word,'is_nar'] for word in sent_tokens_db.index]
#Creating the train and test set
X = [sent2features(sent_idx) for sent_idx in sent_tokens_db['sent_idx'].unique()[:1]]
# y = [sent2labels(s) for s in sentences]
# X = [sent2features(1)]

In [87]:
X

[[{'word': 'טוב',
   'postag': 'NNT',
   'gen': nan,
   'num': nan,
   'suf_per': nan,
   'per': nan,
   'tense': nan,
   'BOS': True,
   '+1:word': 'אהלן',
   '+1:postag': 'INTJ',
   '+1:gen': nan,
   '+1:num': nan,
   '+1:suf_per': nan,
   '+1:per': nan,
   '+1:tense': nan},
  {'word': 'אהלן',
   'postag': 'INTJ',
   'gen': nan,
   'num': nan,
   'suf_per': nan,
   'per': nan,
   'tense': nan,
   '-1:word': 'טוב',
   '-1:postag': 'NNT',
   '-1:gen': nan,
   '-1:num': nan,
   '-1:suf_per': nan,
   '-1:per': nan,
   '-1:tense': nan,
   '+1:word': 'נ',
   '+1:postag': 'NNP',
   '+1:gen': nan,
   '+1:num': nan,
   '+1:suf_per': nan,
   '+1:per': nan,
   '+1:tense': nan},
  {'word': 'נ',
   'postag': 'NNP',
   'gen': nan,
   'num': nan,
   'suf_per': nan,
   'per': nan,
   'tense': nan,
   '-1:word': 'אהלן',
   '-1:postag': 'INTJ',
   '-1:gen': nan,
   '-1:num': nan,
   '-1:suf_per': nan,
   '-1:per': nan,
   '-1:tense': nan,
   '-2:word': 'טוב',
   '-2:postag': 'NNT',
   '-2:gen': nan,
 

In [193]:
def seq2features(seq_idx):
    seq = []
    for s in range(seq_idx,seq_idx+seq_len):
         seq.extend(sent2features(s))
    return seq

def seq2labels(seq_idx):
    seq = []
    for s in range(seq_idx,seq_idx+seq_len):
        label = sent2labels(s)
        seq.extend([label for i in range(vec_dim)]) # 2 - for 2 more featues EOS and BOS
    return seq

In [195]:
X[0], y[0]

(['v0',
  'v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'v7',
  'v8',
  'v9',
  'v10',
  'v11',
  'v12',
  'v13',
  'v14',
  'v15',
  'v16',
  'v17',
  'v18',
  'v19',
  'v20',
  'v21',
  'v22',
  'v23',
  'v24',
  'v25',
  'v26',
  'v27',
  'v28',
  'v29',
  'v30',
  'v31',
  'v32',
  'v33',
  'v34',
  'v35',
  'v36',
  'v37',
  'v38',
  'v39',
  'v40',
  'v41',
  'v42',
  'v43',
  'v44',
  'v45',
  'v46',
  'v47',
  'v48',
  'v49',
  'v50',
  'v51',
  'v52',
  'v53',
  'v54',
  'v55',
  'v56',
  'v57',
  'v58',
  'v59',
  'v60',
  'v61',
  'v62',
  'v63',
  'v64',
  'v65',
  'v66',
  'v67',
  'v68',
  'v69',
  'v70',
  'v71',
  'v72',
  'v73',
  'v74',
  'v75',
  'v76',
  'v77',
  'v78',
  'v79',
  'v80',
  'v81',
  'v82',
  'v83',
  'v84',
  'v85',
  'v86',
  'v87',
  'v88',
  'v89',
  'v90',
  'v91',
  'v92',
  'v93',
  'v94',
  'v95',
  'v96',
  'v97',
  'v98',
  'v99',
  'BOS',
  'v0',
  'v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'v7',
  'v8',
  'v9',
  'v10',
  'v11',
  '

In [196]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [197]:
len(X_train), len(y_train), len(X_test), len(y_test)

(469, 469, 231, 231)

In [198]:
# from sklearn.metrics import make_scorer
# from sklearn_crfsuite import metrics
# f1_scorer = make_scorer(metrics.flat_f1_score, average='macro') 
from sklearn.model_selection import cross_validate

crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
#     all_possible_transitions=True,
)
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
predictions = crf.predict(X_test)
# pred = cross_val_predict(estimator=crf, X=X_train, y=y_train, cv=5)
# scores = cross_validate(crf, X, y, scoring="f1_macro", cv=5)
# print(scores.test_score)

ValueError: The numbers of items and labels differ: |x| = 402, |y| = 408

In [171]:
predictions
# feature_utils.get_prediction_report(y_test,predictions)

[['v0',
  'v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'v7',
  'v8',
  'v9',
  'v10',
  'v11',
  'v12',
  'v13',
  'v14',
  'v15',
  'v16',
  'v17',
  'v18',
  'v19',
  'v20',
  'v21',
  'v22',
  'v23',
  'v24',
  'v25',
  'v26',
  'v27',
  'v28',
  'v29',
  'v30',
  'v31',
  'v32',
  'v33',
  'v34',
  'v35',
  'v36',
  'v37',
  'v38',
  'v39',
  'v40',
  'v41',
  'v42',
  'v43',
  'v44',
  'v45',
  'v46',
  'v47',
  'v48',
  'v49',
  'v50',
  'v51',
  'v52',
  'v53',
  'v54',
  'v55',
  'v56',
  'v57',
  'v58',
  'v59',
  'v60',
  'v61',
  'v62',
  'v63',
  'v64',
  'v65',
  'v66',
  'v67',
  'v68',
  'v69',
  'v70',
  'v71',
  'v72',
  'v73',
  'v74',
  'v75',
  'v76',
  'v77',
  'v78',
  'v79',
  'v80',
  'v81',
  'v82',
  'v83',
  'v84',
  'v85',
  'v86',
  'v87',
  'v88',
  'v89',
  'v90',
  'v91',
  'v92',
  'v93',
  'v94',
  'v95',
  'v96',
  'v97',
  'v98',
  'v99',
  'BOS',
  'v0',
  'v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'v7',
  'v8',
  'v9',
  'v10',
  'v11',
  '

In [51]:
sent_vec_db

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,is_nar,sent_len
0,-0.114521,0.000108,0.036471,-0.024207,0.002051,0.014730,-0.104159,0.058651,0.005404,0.056529,...,-0.002414,0.039244,-0.094238,0.025045,0.097398,-0.004409,0.041561,-0.049388,0.0,9.0
1,-0.013333,0.000648,0.013545,-0.001843,0.037912,0.044653,-0.033737,0.020353,-0.039255,0.040033,...,0.038335,0.013976,-0.062025,-0.025751,-0.032749,0.038144,-0.012054,0.083821,0.0,7.0
2,-0.049959,0.006207,0.019215,-0.013564,-0.006140,-0.020194,-0.021590,0.033078,0.043775,0.076063,...,-0.021449,-0.031051,-0.009857,0.012590,0.040845,-0.031004,0.001470,-0.049673,0.0,6.0
3,-0.009683,0.047087,0.037459,0.019149,0.005652,-0.042241,0.013978,0.052192,-0.023080,0.002327,...,-0.029806,0.033418,-0.029150,0.014673,0.022372,0.058281,0.031803,-0.026994,0.0,18.0
4,0.060957,0.005640,-0.018319,-0.002478,0.010457,-0.024910,0.091064,0.000475,0.066396,-0.038925,...,-0.024806,-0.046600,-0.026737,-0.015007,0.005086,0.030338,0.038523,-0.014919,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,0.044579,0.024742,0.028893,-0.060388,0.039829,0.025243,-0.117620,-0.028324,0.013425,-0.002992,...,-0.027126,-0.003454,-0.037883,0.018410,0.046375,0.052125,-0.023152,0.060160,0.0,3.0
2835,0.017902,0.038459,-0.016246,-0.003681,-0.026354,0.042227,0.019360,0.023875,-0.001420,0.015527,...,-0.022408,0.012324,-0.014626,0.004259,0.006237,0.035307,0.027554,-0.016158,0.0,30.0
2836,0.013650,0.023375,0.017228,0.000760,0.006687,0.010800,-0.005754,0.013923,0.009729,0.018878,...,-0.015082,0.006376,-0.010457,0.021521,-0.011850,0.022129,0.016181,0.004594,0.0,37.0
2837,0.009727,0.017978,0.027529,0.006578,0.020192,-0.020603,0.027635,0.017854,0.040842,0.017801,...,-0.019581,-0.002811,-0.012890,0.033547,0.007128,-0.002388,0.006980,-0.007728,0.0,23.0
